## This script is used for the prepraring dataset for the finetuning DNABERT for SARS-CoV-2 virant clasiification 

In [1]:
import os, tarfile
import glob, time, csv, zipfile, gzip
import pandas as pd
import itertools as it
import sys

In [2]:
os.getcwd()

'/Users/pratik/Documents/GitHub/Postdoc/preprocessing'

In [3]:
variant_dict = {
    "alpha": 0,
    "beta": 1,
    "delta": 2,
    "gamma": 3,
    "eta":4,
    "iota":5,
    "kappa":6,
    "lambda":7,
    "zeta":8
}

In [4]:
variant_dict['alpha']

0

In [5]:
#For Mac
parent_dir_path = "/Users/pratik/Box/Datasets/work_dir/SARS_finetuned/"
#For Windows
#parent_dir_path = "/home/pratik/Work_directory/SARS_finetuned/GISAID"
os.chdir(parent_dir_path)

### Function for creating non-overlappingg chunks with starting position zero and other mentioned starting position

In [11]:
def create_non_overlap_genome(sequence, chunk_size):
    chunks = []
    starting_position_list = [0, 500, 800]
    for position in starting_position_list:
        temp_chunks = [sequence[i:i+chunk_size] for i in range(position, len(sequence), chunk_size)]
        if(len(temp_chunks[-1])<chunk_size):
            del temp_chunks[-1]
        chunks.extend(temp_chunks)
    last_chunk = sequence[-chunk_size:]
    chunks.append(last_chunk)
    return(chunks)

### Finetuning data preparation

In [12]:
def finetune_data_preparation(file_lines, label):
    final_list = []
    for key,group in it.groupby(file_lines,lambda line: line.startswith('>')):
        if not key:
            #count= count+1
            group = list(group)
            stripped_group = list(map(lambda s: s.strip(), group))
            concat_group = ' '.join(stripped_group) 
            concat_group = concat_group.replace(" ","")
            temp_list= create_non_overlap_genome(concat_group, 1000)
            final_list.extend(temp_list)
    temp_df= pd.DataFrame(final_list, columns= ['Sequence'])
    print ("temp", temp_df.shape)
    temp_df = temp_df.drop_duplicates().reset_index(drop=True)
    temp_df['Label']= label
    return (temp_df)

## Remove contradict entries and save in a `.csv` file

In [13]:
def remv_contradict_entries(g, directory):
    print (g.shape)
    df1= g.groupby('Sequence')
    df2= df1.filter(lambda x: len(x) == 1).reset_index(drop=True)
    print (df2.groupby('Label').count())
    df2.to_csv("../1000_chunk_size/Finetuned_SARS_"+directory+".csv", index=False)

In [14]:
for directory in os.listdir():
    count =0
    print (directory)
    input()
    final_df= pd.DataFrame(columns= ['Sequence','Label' ])
    current_path= os.path.join(parent_dir_path,directory)
    #print (current_path)
    folders = os.listdir(directory)
    for subfolder in folders:
        path = os.path.join(current_path, subfolder)
        print (path)
        files= os.listdir(path)
        for file in files:
            if file.endswith("tar"):
                file_path= os.path.join(path, file)
                contents = tarfile.open(file_path)
                #print(contents.getnames())
                for file_name, file_object in zip(contents.getnames(), contents.getmembers()):
                    if file_name.endswith('fasta'):
                        print(file_name)
                        f=contents.extractfile(file_object)
                        file_content=f.readlines()
                        file_content_str = [x.decode() for x in file_content]
                        label = variant_dict[subfolder]
                        temp_df = finetune_data_preparation(file_content_str, label)
                        final_df = final_df.append(temp_df, ignore_index=True)
                        print ("Final",final_df.shape)
    remv_contradict_entries(final_df, directory)

Africa


/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/beta
1629316238802.sequences.fasta
temp (430471, 1)
Final (54038, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/zeta
1629316826903.sequences.fasta
temp (873, 1)
Final (54341, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/eta
1629317297200.sequences.fasta
temp (20001, 1)
Final (61350, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/lambda
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/iota
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/delta
1629316615035.sequences.fasta
temp (91926, 1)
Final (80658, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/kappa
1629317487078.sequences.fasta
temp (786, 1)
Final (81304, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Africa/alpha
1629314973953.sequences.fasta
temp (99805, 1)
Final (97774, 2)
(97774, 2)
       Sequence
Label          
0         13686
1         50033
2         16488
4          5488
6           351
8 

/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/beta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/zeta
1629323030691.sequences.fasta
temp (140605, 1)
Final (17551, 2)
1629322790999.sequences.fasta
temp (22712, 1)
Final (23884, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/eta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/lambda
1629322196621.sequences.fasta
temp (5343, 1)
Final (26389, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/iota
1629323180991.sequences.fasta
temp (17899, 1)
Final (31092, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/delta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/gamma
1629321882420.sequences.fasta
temp (428385, 1)
Final (69603, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/kappa
1629323292487.sequences.fasta
temp (62989, 1)
Final (79740, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Europe/alpha
(79740, 2)
       Sequence
Label          
3         35224

/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/beta
1629323731996.sequences.fasta
temp (6095, 1)
Final (1949, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/zeta
1629323703456.sequences.fasta
temp (434, 1)
Final (2205, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/eta
1629324106221.sequences.fasta
temp (607, 1)
Final (2539, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/lambda
1629324181209.sequences.fasta
temp (87, 1)
Final (2626, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/iota
1629324129472.sequences.fasta
temp (174, 1)
Final (2713, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/delta
1629323796416.sequences.fasta
temp (49983, 1)
Final (12259, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/gamma
1629323757740.sequences.fasta
temp (610, 1)
Final (12504, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Oceania/kappa
1629324155720.sequences.fasta
temp (6963, 1)
Final (13874, 2)
/home/

/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/beta
1629389299611.sequences.fasta
temp (1572, 1)
Final (725, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/zeta
1629389399049.sequences.fasta
temp (216583, 1)
Final (37413, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/eta
1629389807077.sequences.fasta
temp (87, 1)
Final (37500, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/lambda
1629389882085.sequences.fasta
temp (19133, 1)
Final (44375, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/iota
1629389834686.sequences.fasta
temp (31600, 1)
Final (49553, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/delta
1629389326073.sequences.fasta
temp (29892, 1)
Final (56172, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/kappa
1629389859687.sequences.fasta
temp (87, 1)
Final (56259, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/South America/alpha
16293892336

/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/beta
1629319610891.sequences.fasta
temp (199204, 1)
Final (26659, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/zeta
1629319354850.sequences.fasta
temp (137616, 1)
Final (53067, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/eta
1629319246862.sequences.fasta
temp (43591, 1)
Final (60091, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/lambda
1629318739604.sequences.fasta
temp (11021, 1)
Final (64587, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/iota
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/delta
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/kappa
1629319061098.sequences.fasta
temp (37008, 1)
Final (72568, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/North America/alpha
1630017471230.sequences.fasta
temp (66259, 1)
Final (87224, 2)
(87224, 2)
       Sequence
Label          
0         12288
1    

/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/beta
1629321017543.sequences.fasta
temp (177323, 1)
Final (30374, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/zeta
1629320920540.sequences.fasta
temp (1578, 1)
Final (31649, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/eta
1629321116083.sequences.fasta
temp (27574, 1)
Final (43553, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/lambda
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/iota
1629321359081.sequences.fasta
temp (1493, 1)
Final (44831, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/delta
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/gamma
1629320543386.sequences.fasta
temp (15102, 1)
Final (48614, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/kappa
1629321299449.sequences.fasta
temp (230172, 1)
Final (89284, 2)
/home/pratik/Work_directory/SARS_finetuned/GISAID/Asia/alpha
(89284, 2)
       Sequence
Label          
1         26841
3          2725
4 